# Facebook Friend Recommendation Part 2 Featurization

In [37]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import networkx as nx
import math
import os
import pickle
from tqdm import tqdm
from pandas import HDFStore,DataFrame
from pandas import read_hdf

In [2]:
train_graph = nx.read_edgelist("data/after_eda/train_pos_after_eda.csv",delimiter = ",",create_using = nx.DiGraph(),nodetype = int)
print(nx.info(train_graph))

Name: 
Type: DiGraph
Number of nodes: 1780722
Number of edges: 7550015
Average in degree:   4.2399
Average out degree:   4.2399


# 2. Similarity measures

## 2.1 Jaccard Distance:
http://www.statisticshowto.com/jaccard-index/

\begin{equation}
j = \frac{|X\cap Y|}{|X \cup Y|} 
\end{equation}

### How Jaccard Distance works

<img src="jaccard.png" width="400" height="200">

#### Lets: Create 2 Sets X & Y
        X is set of followers User 0 has -- > {u2, u3, u4}
        Y is set of followers User 1 has -- > {u3, u4, u5}
    
\begin{equation}
j = \frac{|X\cap Y|}{|X \cup Y|}  = 2 / 4
\end{equation}

       Note: Higher the Jaccard Distance b/w U0 and U1 chances of having edge b/w them is higher

In [3]:
## Jaccard for followees
def jaccard_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a))) == 0  | len(set(train_graph.successors(b))) == 0:
            ## sucessors = +1
            return 0
        sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
                                    (len(set(train_graph.successors(a)).union(set(train_graph.successors(b)))))
    except:
        return 0
    return sim    

In [4]:
print(jaccard_for_followees(111,18324))

0.0


In [5]:
print(jaccard_for_followees(1131,324))

0.0


In [6]:
#for followers
def jaccard_for_followers(a,b):
    try:
        if len(set(train_graph.predecessors(a))) == 0  | len(set(g.predecessors(b))) == 0:
            return 0
        sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                 (len(set(train_graph.predecessors(a)).union(set(train_graph.predecessors(b)))))
        return sim
    except:
        return 0

In [7]:
#node 1635354 not in graph 
print(jaccard_for_followees(456,699))

0.0


## 2.2 Cosine distance

\begin{equation}
CosineDistance = \frac{|X\cap Y|}{|X|\cdot|Y|} 
\end{equation}

In [8]:
#for followees
def cosine_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a))) == 0  | len(set(train_graph.successors(b))) == 0:
            return 0
        sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
                                    (math.sqrt(len(set(train_graph.successors(a)))*len((set(train_graph.successors(b))))))
        return sim
    except:
        return 0

In [9]:
print(cosine_for_followees(273084,1505602))

0.0


In [10]:
#for followeers
def cosine_for_followers(a,b):
    try:
        
        if len(set(train_graph.predecessors(a))) == 0  | len(set(train_graph.predecessors(b))) == 0:
            return 0
        sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                     (math.sqrt(len(set(train_graph.predecessors(a))))*(len(set(train_graph.predecessors(b)))))
        return sim
    except:
        return 0

In [11]:
print(cosine_for_followers(2,470294))

0.02886751345948129


## 2.3 Page Rank
### source : https://en.wikipedia.org/wiki/PageRank

<img src="pagerank.jpg" width="400" height="200">

### How page rank works?
    Given a Directed graph--- page rank algorithm --- > gives each vertex/node (ui) ---> Score 
    That score represent how imp the vertex is
    To compute page rank using Python we will use inbilt library networkx

In [12]:
if not os.path.isfile('data/fea_sample/page_rank.p'):
    pr = nx.pagerank(train_graph, alpha=0.85)
    pickle.dump(pr,open('data/fea_sample/page_rank.p','wb'))
else:
    pr = pickle.load(open('data/fea_sample/page_rank.p','rb'))

In [13]:
print('min',pr[min(pr, key=pr.get)])

min 1.6556497245737814e-07


In [44]:
#for imputing to nodes which are not there in Train data
mean_pr = float(sum(pr.values())) / len(pr)
print(mean_pr)

5.615699699389075e-07


## 2.4 Shortest Path between 2 nodes

<img src="shortest_path.png" width="400" height="200">

#### Shortest path from Ui to Uj:
    From Ui to Uj ---> [2,Uj]     length = 2
    From Ui to Uj ---> [3,4,5,Uj] length = 4
#### So we will take shortest path from Ui to Uj with length = 2   
#### Note : while computing shortest path if there is direct edge b/w ui and uj {i.e. length = 1} then we will simply remove it and find shortest path

In [14]:
def compute_shortest_path_length(a,b):
    p = -1
    try:
        if train_graph.has_edge(a,b):    # ie if it already has edge we will remove it and find shortest path
            train_graph.remove_edge(a,b)
            p = nx.shortest_path_length(train_graph,source = a, target = b)  # compute it now
            train_graph.add_edge(a,b)        # add them
        else:
            p = nx.shortest_path_length(train_graph,source = a,target = b)   # else simply add them
        return p
    except:
        return -1   

In [15]:
compute_shortest_path_length(77697, 826021)

10

In [16]:
compute_shortest_path_length(669354,1635354)

-1

## 2.5 Adamic/ Adar Index
$$A(x,y)=\sum_{u \in N(x) \cap N(y)}\frac{1}{log(|N(u)|)}$$

#### Source: https://en.wikipedia.org/wiki/Adamic/Adar_index

### Some imp points regrading Adar Index
    1. From given formula of Adar Index N(x) and N(y) denotes as Neigbour of vertex x & y
    2. N(x) intersection N(y) = common vertex between them

### Q . Why we use $\frac{1}{log(|N(u)|)}$ in formula ? 
     Lets from N(x) intersction N(y) we got 2 common vertex (U1 ,U2)
     
     Case 1: Out of that U1 has high number of followers.
     >> So there may be chance that he will be a celebrity.
     >> So lets U1 is hollywood actor Johnny depp where me (x) from India and (y) some guy from Canada follows him.
     >> this does not means that i should also follow that (y) from Canada. 
     >> thats y when U1 is high we use 1/log where log is monotonic func to reduce the value when its larger
     >> 1 / log (40000)  = 0.21
     
     Case 2: Where U2 is normal guy with less number of follwers so the followers range will be from nearby locality
     >> so there might be chance that x might follow y
     >> 1 / log (40) = 0.62

In [17]:
def calc_adar_in(a,b):
    sum = 0
    try:
        n = list(set(train_graph.successors(a).intersection(set(train_graph.successors(b)))))  
        ## sucessors =  (followees) i.e we have to check how many are following a & b
        if len(n)!=0:   # that means there are some common vertex so now calculate sum by using formula as discussed above
            for i in n:
                sum = sum(1/ np.log10(len(list(train_graph.predecessors(i)))))
                #predecessors = (followers) i.e we have to check each user having how many number of followers 
                # check if he is celebrity or not and then use 1/log to minimize it
            return sum
        else:
            return 0     
    except:
        return 0

In [18]:
calc_adar_in(4533,454343)

0

In [19]:
calc_adar_in(111,12311)

0

## 2.6 Follow Back

#### Lets task: find directed edge between Ui and Uj
    But there is already an directed edge b/w Uj and Ui (i.e Uj is following Ui) 
    Then there wil be high chance that Ui will follow Uj  (i.e Ui will follow Uj)

In [20]:
def follow_back(a,b):  # Ui and Uj
    if train_graph.has_edge(b,a):    ## check if Uj and Ui has directed edge
        return 1
    else:
        return 0

In [21]:
follow_back(1,189226)

1

In [22]:
follow_back(1323,342)

0

# 3. Featurization

### 3.1 Reading a sample of Data from both train and test

In [23]:
import random
if os.path.isfile('data/after_eda/train_after_eda.csv'):
    filename = "data/after_eda/train_after_eda.csv"
    n_train =  15100028
    s = 100000 #desired sample size for this case studies
    skip_train = sorted(random.sample(range(1,n_train+1),n_train-s))
    #https://stackoverflow.com/a/22259008/4084039

In [24]:
if os.path.isfile('data/after_eda/train_after_eda.csv'):
    filename = "data/after_eda/test_after_eda.csv"
    n_test = 3775006
    s = 50000 #desired sample size for this case study
    skip_test = sorted(random.sample(range(1,n_test+1),n_test-s))
    #https://stackoverflow.com/a/22259008/4084039

In [25]:
print("Number of rows in the train data file:", n_train)
print("Number of rows we are going to elimiate in train data are",len(skip_train))
print("Number of rows in the test data file:", n_test)
print("Number of rows we are going to elimiate in test data are",len(skip_test))

Number of rows in the train data file: 15100028
Number of rows we are going to elimiate in train data are 15000028
Number of rows in the test data file: 3775006
Number of rows we are going to elimiate in test data are 3725006


In [26]:
df_final_train = pd.read_csv('data/after_eda/train_after_eda.csv',skiprows=skip_train, names=['source_node', 'destination_node'])
df_final_train['indicator_link'] = pd.read_csv('data/train_y.csv',skiprows=skip_train,  names=['indicator_link'])
print("Our train matrix size ",df_final_train.shape)
df_final_train.head(2)

Our train matrix size  (100002, 3)


,source_node,destination_node,indicator_link
0,273084,1505602,1
1,1050142,773379,1


In [27]:
df_final_test = pd.read_csv('data/after_eda/test_after_eda.csv', skiprows=skip_test, names=['source_node', 'destination_node'])
df_final_test['indicator_link'] = pd.read_csv('data/test_y.csv', skiprows=skip_test, names=['indicator_link'])
print("Our test matrix size ",df_final_test.shape)
df_final_test.head(2)

Our test matrix size  (50002, 3)


,source_node,destination_node,indicator_link
0,848424,784690,1
1,1023305,1027793,1


## 3.1 Weight Feature

\begin{equation}
W = \frac{1}{\sqrt{1+|X|}}
\end{equation}

#### Lets Ui (is a Famous person) and Uj (Normal person)
    Ui ---> 1 million incoming(i.e followers)
    Uj ---> ony 100 incomming (i.e followers)
    By using weight feature we can detect who is Celebrity & who is normal person
    # Ui = 1 / sqrt(1 + 1 million)
    # Uj = 1 / sqrt(1 + 100)
    So in this case Ui << Uj


In [33]:
## weight for both incoming(followers) & outcoming (followees)

Weight_in = {}
Weight_out = {}

for i in tqdm(train_graph.nodes()):
    s1 = set(train_graph.predecessors(i))
    w_in = 1.0/(np.sqrt(1+len(s1)))
    Weight_in[i] = w_in
    
    s2 = set(train_graph.successors(i))
    w_out = 1.0/(np.sqrt(1+len(s2)))
    Weight_out[i] = w_out

# impute with mean
mean_weight_in = np.mean(list(Weight_in.values()))  
mean_weight_out = np.mean(list(Weight_out.values()))  

100%|██████████████████████████████| 1780722/1780722 [25:45<00:00, 1152.43it/s]


## 4.1 Add all Features
__we will create these each of these features for both train and test data points__
<ol>
<li>jaccard_followers</li>
<li>jaccard_followees</li>
<li>cosine_followers</li>
<li>cosine_followees</li>
<li>num_followers_s</li>
<li>num_followees_s</li>
<li>num_followers_d</li>
<li>num_followees_d</li>
<li>inter_followers</li>
<li>inter_followees</li>
</ol>

In [34]:
if not os.path.isfile('data/fea_sample/storage_sample_stage1.h5'):
    #mapping jaccrd followers to train and test data
    df_final_train['jaccard_followers'] = df_final_train.apply(lambda row:
                                            jaccard_for_followers(row['source_node'],row['destination_node']),axis=1)
    df_final_test['jaccard_followers'] = df_final_test.apply(lambda row:
                                            jaccard_for_followers(row['source_node'],row['destination_node']),axis=1)

    #mapping jaccrd followees to train and test data
    df_final_train['jaccard_followees'] = df_final_train.apply(lambda row:
                                            jaccard_for_followees(row['source_node'],row['destination_node']),axis=1)
    df_final_test['jaccard_followees'] = df_final_test.apply(lambda row:
                                            jaccard_for_followees(row['source_node'],row['destination_node']),axis=1)
    

        #mapping jaccrd followers to train and test data
    df_final_train['cosine_followers'] = df_final_train.apply(lambda row:
                                            cosine_for_followers(row['source_node'],row['destination_node']),axis=1)
    df_final_test['cosine_followers'] = df_final_test.apply(lambda row:
                                            cosine_for_followers(row['source_node'],row['destination_node']),axis=1)

    #mapping jaccrd followees to train and test data
    df_final_train['cosine_followees'] = df_final_train.apply(lambda row:
                                            cosine_for_followees(row['source_node'],row['destination_node']),axis=1)
    df_final_test['cosine_followees'] = df_final_test.apply(lambda row:
                                            cosine_for_followees(row['source_node'],row['destination_node']),axis=1)

In [35]:
def compute_features_stage1(df_final):
    #calculating no of followers followees for source and destination
    #calculating intersection of followers and followees for source and destination
    num_followers_s=[]
    num_followees_s=[]
    num_followers_d=[]
    num_followees_d=[]
    inter_followers=[]
    inter_followees=[]
    for i,row in df_final.iterrows():
        try:
            s1=set(train_graph.predecessors(row['source_node']))
            s2=set(train_graph.successors(row['source_node']))
        except:
            s1 = set()
            s2 = set()
        try:
            d1=set(train_graph.predecessors(row['destination_node']))
            d2=set(train_graph.successors(row['destination_node']))
        except:
            d1 = set()
            d2 = set()
        num_followers_s.append(len(s1))
        num_followees_s.append(len(s2))

        num_followers_d.append(len(d1))
        num_followees_d.append(len(d2))

        inter_followers.append(len(s1.intersection(d1)))
        inter_followees.append(len(s2.intersection(d2)))
    
    return num_followers_s, num_followers_d, num_followees_s, num_followees_d, inter_followers, inter_followees

In [38]:
if not os.path.isfile('data/fea_sample/storage_sample_stage1.h5'):
    df_final_train['num_followers_s'], df_final_train['num_followers_d'], \
    df_final_train['num_followees_s'], df_final_train['num_followees_d'], \
    df_final_train['inter_followers'], df_final_train['inter_followees']= compute_features_stage1(df_final_train)
    
    df_final_test['num_followers_s'], df_final_test['num_followers_d'], \
    df_final_test['num_followees_s'], df_final_test['num_followees_d'], \
    df_final_test['inter_followers'], df_final_test['inter_followees']= compute_features_stage1(df_final_test)
    
    hdf = HDFStore('data/fea_sample/storage_sample_stage1.h5')
    hdf.put('train_df',df_final_train, format='table', data_columns=True)
    hdf.put('test_df',df_final_test, format='table', data_columns=True)
    hdf.close()
else:
    df_final_train = read_hdf('data/fea_sample/storage_sample_stage1.h5', 'train_df',mode='r')
    df_final_test = read_hdf('data/fea_sample/storage_sample_stage1.h5', 'test_df',mode='r')

## 4.1 Add all Features
__we will add some more features for both train and test data points__
<ol>
<li>Adar Index</li>
<li>is following back</li>
<li>shortest path between source and destination</li>
</ol>

In [40]:
if not os.path.isfile('data/fea_sample/storage_sample_stage2.h5'):
    #mapping adar index on train
    df_final_train['adar_index'] = df_final_train.apply(lambda row: calc_adar_in(row['source_node'],row['destination_node']),axis=1)
    #mapping adar index on test
    df_final_test['adar_index'] = df_final_test.apply(lambda row: calc_adar_in(row['source_node'],row['destination_node']),axis=1)

    #--------------------------------------------------------------------------------------------------------
    #mapping followback or not on train
    df_final_train['follow_back'] = df_final_train.apply(lambda row: follow_back(row['source_node'],row['destination_node']),axis=1)

    #mapping followback or not on test
    df_final_test['follow_back'] = df_final_test.apply(lambda row: follow_back(row['source_node'],row['destination_node']),axis=1)

    #--------------------------------------------------------------------------------------------------------
    #mapping shortest path on train 
    df_final_train['shortest_path'] = df_final_train.apply(lambda row: compute_shortest_path_length(row['source_node'],row['destination_node']),axis=1)
    #mapping shortest path on test
    df_final_test['shortest_path'] = df_final_test.apply(lambda row: compute_shortest_path_length(row['source_node'],row['destination_node']),axis=1)

    hdf = HDFStore('data/fea_sample/storage_sample_stage2.h5')
    hdf.put('train_df',df_final_train, format='table', data_columns=True)
    hdf.put('test_df',df_final_test, format='table', data_columns=True)
    hdf.close()
else:
    df_final_train = read_hdf('data/fea_sample/storage_sample_stage2.h5', 'train_df',mode='r')
    df_final_test = read_hdf('data/fea_sample/storage_sample_stage2.h5', 'test_df',mode='r')

## 4.2 Add all Features
__we will add some more features for both train and test data points__
<ol>
<li>Weight in and out</li>
<li>Page Rank</li>
</ol>

In [42]:
if not os.path.isfile('data/fea_sample/storage_sample_stage3.h5'):
    #mapping to pandas train
    df_final_train['weight_in'] = df_final_train.destination_node.apply(lambda x: Weight_in.get(x,mean_weight_in))
    df_final_train['weight_out'] = df_final_train.source_node.apply(lambda x: Weight_out.get(x,mean_weight_out))

    #mapping to pandas test
    df_final_test['weight_in'] = df_final_test.destination_node.apply(lambda x: Weight_in.get(x,mean_weight_in))
    df_final_test['weight_out'] = df_final_test.source_node.apply(lambda x: Weight_out.get(x,mean_weight_out))

In [45]:
if not os.path.isfile('data/fea_sample/storage_sample_stage3.h5'):
    
    #page rank for source and destination in Train and Test
    #if anything not there in train graph then adding mean page rank 
    df_final_train['page_rank_s'] = df_final_train.source_node.apply(lambda x:pr.get(x,mean_pr))
    df_final_train['page_rank_d'] = df_final_train.destination_node.apply(lambda x:pr.get(x,mean_pr))

    df_final_test['page_rank_s'] = df_final_test.source_node.apply(lambda x:pr.get(x,mean_pr))
    df_final_test['page_rank_d'] = df_final_test.destination_node.apply(lambda x:pr.get(x,mean_pr))
    
    hdf = HDFStore('data/fea_sample/storage_sample_stage3.h5')
    hdf.put('train_df',df_final_train, format='table', data_columns=True)
    hdf.put('test_df',df_final_test, format='table', data_columns=True)
    hdf.close()
else:
    df_final_train = read_hdf('data/fea_sample/storage_sample_stage3.h5', 'train_df',mode='r')
    df_final_test = read_hdf('data/fea_sample/storage_sample_stage3.h5', 'test_df',mode='r')

In [46]:
df_final_train.columns

Index(['source_node', 'destination_node', 'indicator_link',
       'jaccard_followers', 'jaccard_followees', 'cosine_followers',
       'cosine_followees', 'num_followers_s', 'num_followers_d',
       'num_followees_s', 'num_followees_d', 'inter_followers',
       'inter_followees', 'adar_index', 'follow_back', 'shortest_path',
       'weight_in', 'weight_out', 'page_rank_s', 'page_rank_d'],
      dtype='object')

In [48]:
df_final_train.sample(5)

,source_node,destination_node,indicator_link,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,adar_index,follow_back,shortest_path,weight_in,weight_out,page_rank_s,page_rank_d
61811,1561712,1459191,0,0,0.0,0.0,0.0,11,0,7,1,0,0,0,0,-1,1.000000,0.353553,5.885580e-07,1.655650e-07
38288,1394779,1367495,1,0,0.0,0.0,0.0,32,3,40,5,0,0,0,1,5,0.500000,0.156174,2.517703e-06,2.132346e-07
54957,883502,1281630,0,0,0.0,0.0,0.0,2,19,6,15,0,0,0,0,7,0.223607,0.377964,2.550652e-07,2.016170e-06
17370,1628873,163367,1,0,0.0,0.0,0.0,7,1,6,1,0,0,0,1,-1,0.707107,0.377964,1.161906e-06,2.451207e-07
30960,648021,867107,1,0,0.0,0.0,0.0,8,3,5,3,0,0,0,1,4,0.500000,0.408248,6.394345e-07,4.883340e-07


#### So orignally we had only 2 features [i.e Source Node, Destination Node], but after applying couple of feature engineering techniques we got 20 features so now we can apply Machine learning techniques further 